# [모듈 01] Amazon S3 버킷에서 SageMaker Feature Store로 자동화된 데이터 변환 및 수집

이 노트북은 다음과 같은 작업을 합니다.

1. 아키텍처 개요
2. 전제 조건
3. 기본 변수 로딩
    - S3 data prefix to monitor : CSV 가 업로드가 되는 S3 Prefix
    - Data Wrangler flow URL
    - Data Wrangler output name
    - Feature group name
4. 세이지 메이커 사용자 템플릿을 통한 프로젝트 생성
5. 세이지 메이커 사용자 템플릿을 통한 프로젝트 분석 (데이타 수집 프로젝트)
6. 자동화 파이프라인 테스트
    - 지정된 S3 Prefix 에 CSV 를 업로딩하여 파이브라인을 자동으로 실행
7. 피쳐 스토어에서 데이터 확인
---

# 1. 아키텍처 개요
이 노트북은 [AWS 서비스 카탈로그](https://aws.amazon.com/servicecatalog), [SageMaker 프로젝트](https://docs.aws.amazon.com/sagemaker/latest/dg/sagemaker) 사용 방법을 보여줍니다. -projects-whatis.html) 및 [Pipelines](https://aws.amazon.com/sagemaker/pipelines/)를 사용하여 SageMaker Studio에서 재사용 가능하고 이식 가능한 구성 요소를 생성합니다.

이 프로젝트는 S3 버킷에 업로드된 새 데이터 파일에서 트리거되는 [SageMaker Feature Store](https://aws.amazon.com/sagemaker/feature-store/)로의 기능 변환 및 수집을 자동화합니다. SageMaker 프로젝트는 필요한 모든 구성 요소를 생성하고 리소스 간의 모든 권한 및 링크를 설정합니다.

<img src="../design/feature-store-ingestion-pipeline.drawio.svg" style="background-color:white;" alt="solution overview" width="1000"/>

# 2. 전제 조건

## 2.1. 기본 리소스 생성 
- SageMaker 프로젝트 배포를 진행하려면 먼저 다음 리소스를 생성해야 합니다.
- 이 모든 작업은 [`00-setup` 노트북](00-setup.ipynb)에서 수행됩니다. 이 노트북을 실행하기 전에 설정 노트북을 실행했는지 확인하십시오.
    - 출력 노드를 포함하는 Data Wrangler `.flow` 파일. `.flow` 파일은 지정된 S3 접두사에 업로드해야 합니다.
    - 데이터에서 추출한 피쳐를 저장할 피쳐 그룹
    - SageMaker 프로젝트 포트폴리오 -> [초기 설정]으로 완료(../README.md#deploy-sagemaker-project-portfolio)
    - 새 데이터 파일이 업로드될 S3 버킷



## 2.2. AmazonSageMakerServiceCatalogProductsLaunchRole 에 권한 추가
<div class="alert alert-info"> 💡 <strong> AmazonSageMakerServiceCatalogProductsLaunchRole 에 AWSLambda_FullAccess, IAMFullAccess 추가 함</strong></div>

In [1]:
%%sh

aws iam attach-role-policy \
    --role-name  AmazonSageMakerServiceCatalogProductsLaunchRole \
    --policy-arn arn:aws:iam::aws:policy/AWSLambda_FullAccess

aws iam attach-role-policy \
    --role-name  AmazonSageMakerServiceCatalogProductsLaunchRole \
    --policy-arn arn:aws:iam::aws:policy/IAMFullAccess



# 3. 기본 변수 로딩

In [2]:
import sagemaker
import boto3
import time
import json
import os
from time import gmtime, strftime
from sagemaker.workflow.pipeline import Pipeline
from sagemaker.feature_store.feature_group import FeatureGroup
from sagemaker.session import Session

print(sagemaker.__version__)

2.70.0


In [3]:
# load environment variables from %store
%store -r 

In [4]:
%store

Stored variables and their in-db values:
abalone_dataset_file_name             -> 'abalone.csv'
abalone_dataset_local_url             -> '../dataset/abalone.csv'
data_bucket                           -> 'sagemaker-ap-northeast-2-018763497627'
domain_id                             -> 'd-lnzveulicvk5'
dw_flow_file_url                      -> 's3://sagemaker-ap-northeast-2-018763497627/featur
dw_output_name                        -> 'c8880ed5-b8a0-4375-899b-1c4d86828152.default'
execution_role                        -> 'arn:aws:iam::018763497627:role/mod-6297809195fe48
feature_group_name                    -> 'FG-abalone-07-11-12-33-f12df440'
s3_data_prefix                        -> 'sagemaker-ap-northeast-2-018763497627/feature-sto
s3_flow_prefix                        -> 'sagemaker-ap-northeast-2-018763497627/feature-sto
s3_fs_query_output_prefix             -> 'sagemaker-ap-northeast-2-018763497627/feature-sto
s3_input_data_prefix                  -> 'sagemaker-ap-northeast-2-018763497

In [5]:
try:
    data_bucket
    dw_flow_file_url
    dw_output_name
    feature_group_name
    s3_fs_query_output_prefix
    s3_data_prefix
    s3_flow_prefix
    abalone_dataset_local_url
except NameError:
    print("+++++++++++++++++++++++++++++++++++++++++++++++")
    print("[ERROR] YOU HAVE TO RUN 00-setup.ipynb notebook")
    print("+++++++++++++++++++++++++++++++++++++++++++++++")

In [6]:
# Set the string literals
s3_input_data_prefix = f"{data_bucket}/feature-store-ingestion-pipeline/landing-zone/"
pipeline_name_prefix = "s3-fs-ingest-pipeline"

%store s3_input_data_prefix

Stored 's3_input_data_prefix' (str)


프로젝트 생성을 위한 아래 주요한 변수를 확인 하세요.

In [7]:
print("Project parameters:")
print(f"S3 data prefix to monitor: {s3_input_data_prefix}")
print(f"Data Wrangler flow URL: {dw_flow_file_url}")
print(f"Data Wrangler output name: {dw_output_name}")
print(f"Feature group name: {feature_group_name}")

Project parameters:
S3 data prefix to monitor: sagemaker-ap-northeast-2-018763497627/feature-store-ingestion-pipeline/landing-zone/
Data Wrangler flow URL: s3://sagemaker-ap-northeast-2-018763497627/feature-store-ingestion-pipeline/dw-flow/dw2-flow-07-11-12-33-f12df440.flow
Data Wrangler output name: c8880ed5-b8a0-4375-899b-1c4d86828152.default
Feature group name: FG-abalone-07-11-12-33-f12df440


# 4. 세이지 메이커 사용자 템플릿을 통한 프로젝트 생성
- ⭐ Studio IDE(옵션 1)에서 프로젝트를 생성하거나 이 노트북에서 직접 프로그래밍 방식으로(옵션 2) 프로젝트를 생성할 수 있습니다. 
- 옵션 2는 수동 입력이 필요하지 않으므로 권장됩니다. 
    - 옵션 1은 프로젝트 매개변수에 대한 UX를 시연하기 위해 제공됩니다.

## 4.1. 옵션 1: Studio에서 프로젝트 만들기

1. **SageMaker 리소스** 위젯에서 **프로젝트**를 선택합니다.

<img src="../img/studio-create-project.png" alt="studio-create-project" width="400"/>

2. **조직 템플릿**으로 이동하여 자동화된 변환 및 수집 파이프라인을 위한 프로젝트 템플릿을 선택합니다. **프로젝트 템플릿 선택**을 클릭합니다.

<img src="../img/studio-select-project-template.png" width="800"/>

3. 프로젝트 파라미터를 입력 하세요.
<img src="../img/studio-enter-project-parameters.png" width="800"/>

매개변수는 다음과 같습니다.
- **프로젝트 이름 및 설명**: 프로젝트 이름 및 설명 제공
- **파이프라인 이름 접두사**: 파이프라인 이름에 대한 접두사를 제공하거나 기본값을 그대로 둡니다.
- **파이프라인 설명**: 파이프라인에 대한 설명을 제공하거나 기본값을 그대로 둡니다.
- **S3 접두사**: `s3_input_data_prefix` 변수의 값으로 설정
- **Data Wrangler flow S3 url**: `dw_flow_file_url` 변수 값으로 설정
- **Data Wrangler 출력 이름**: `dw_output_name` 변수의 값으로 설정
- **기능 그룹 이름**: `feature_group_name` 변수의 값으로 설정
- **Lambda 실행 역할**: 람다 함수에 대한 고유한 IAM 역할을 제공하거나 '자동'으로 두어 새 역할을 자동으로 생성합니다.

**프로젝트 만들기**를 클릭합니다.

<div class="alert alert-info"> 💡 <strong> 프로젝트 생성이 완료될 때까지 기다리기 </strong>
</div>
배너 "프로젝트 생성 중...":

<img src="../img/studio-creating-project-banner.png" alt="studio-creating-project-banner" width="500"/>

프로젝트 세부 정보 페이지로 변경됩니다.

<img src="../img/studio-project-created.png" width="800"/>

#### 생성된 프로젝트의 이름과 ID를 가져옵니다.

<div class="alert alert-info"> 💡 <strong> 옵션 1 - Studio IDE에서 프로젝트 생성을 사용하는 경우에만 다음 셀을 실행하십시오. </strong>

In [ ]:
# Get the latest created project
sm = boto3.client("sagemaker")
r = sm.list_projects(SortBy="CreationTime", SortOrder="Descending")

In [ ]:
r

In [ ]:
if r.get("ProjectSummaryList") is None or len(r.get("ProjectSummaryList")) == 0:
    raise Exception("[ERROR]: cannot retrieve the project list!")
    
if r["ProjectSummaryList"][0]["ProjectStatus"] not in ("CreateCompleted"):
    raise Exception("[ERROR]: wait until project creation is completed!")
else:
    project_name = r["ProjectSummaryList"][0]["ProjectName"]
    project_id = r["ProjectSummaryList"][0]["ProjectId"]

### End of Option 1 section
---

## 4.2 옵션 2: 코드로 프로젝트 생성 - 권장
<div class="alert alert-info"> 💡 <strong> Studio IDE를 통해 프로젝트를 만든 경우 이 섹션을 건너뛰세요. </strong>

- [boto3 Python SDK](https://boto3.amazonaws.com/v1/documentation/api/latest/reference/services/sagemaker.html#SageMaker.Client.create_project)를 사용하여 노트북에서 새 프로젝트를 생성하거나 파이썬 코드를 사용하세요..
- 먼저 서비스 카탈로그 CloudFormation 템플릿에서 `ProvisioningArtifactIds` 및 `ProductId`를 가져옵니다.

In [8]:
cf = boto3.client("cloudformation")

r = cf.describe_stacks(StackName="sm-project-sc-portfolio")
r

{'Stacks': [{'StackId': 'arn:aws:cloudformation:ap-northeast-2:018763497627:stack/sm-project-sc-portfolio/a54a3d10-8806-11ec-88e0-0278d35f1cce',
   'StackName': 'sm-project-sc-portfolio',
   'Description': 'Create Service Catalog products as SageMaker project templates for various re-usable components\n',
   'Parameters': [{'ParameterKey': 'SCPortfolioPrincipalRoleArn',
     'ParameterValue': 'arn:aws:iam::018763497627:role/mod-6297809195fe4845-SageMakerExecutionRole-1A1EDDFAJER7W'},
    {'ParameterKey': 'SCProductLaunchRoleArn',
     'ParameterValue': 'AmazonSageMakerServiceCatalogProductsLaunchRole'}],
   'CreationTime': datetime.datetime(2022, 2, 7, 11, 11, 6, 712000, tzinfo=tzlocal()),
   'RollbackConfiguration': {},
   'StackStatus': 'CREATE_COMPLETE',
   'DisableRollback': True,
   'NotificationARNs': [],
   'Capabilities': ['CAPABILITY_NAMED_IAM'],
   'Outputs': [{'OutputKey': 'ProductName',
     'OutputValue': 'Automated Feature Transformation and Ingestion Pipeline v1.0',
    

SageMaker 프로젝트에 대한 매개변수 설정:

In [9]:
sm = boto3.client("sagemaker")

provisioning_artifact_ids = [v for v in r["Stacks"][0]["Outputs"] if v["OutputKey"] == "ProvisioningArtifactIds"][0]["OutputValue"]
product_id = [v for v in r["Stacks"][0]["Outputs"] if v["OutputKey"] == "ProductId"][0]["OutputValue"]
project_name = f"s3-fs-ingest-{strftime('%d-%H-%M-%S', gmtime())}"
project_parameters = [
            {
                'Key': 'PipelineDescription',
                'Value': 'Feature Store ingestion pipeline'
            },
            {
                'Key': 'DataWranglerFlowUrl',
                'Value': dw_flow_file_url
            },
            {
                'Key': 'DataWranglerOutputName',
                'Value': dw_output_name
            },
            {
                'Key': 'S3DataPrefix',
                'Value': s3_input_data_prefix
            },
            {
                'Key': 'FeatureGroupName',
                'Value': feature_group_name
            },
            {
                'Key': 'PipelineNamePrefix',
                'Value': pipeline_name_prefix
            },
        ]

In [10]:
project_parameters

[{'Key': 'PipelineDescription', 'Value': 'Feature Store ingestion pipeline'},
 {'Key': 'DataWranglerFlowUrl',
  'Value': 's3://sagemaker-ap-northeast-2-018763497627/feature-store-ingestion-pipeline/dw-flow/dw2-flow-07-11-12-33-f12df440.flow'},
 {'Key': 'DataWranglerOutputName',
  'Value': 'c8880ed5-b8a0-4375-899b-1c4d86828152.default'},
 {'Key': 'S3DataPrefix',
  'Value': 'sagemaker-ap-northeast-2-018763497627/feature-store-ingestion-pipeline/landing-zone/'},
 {'Key': 'FeatureGroupName', 'Value': 'FG-abalone-07-11-12-33-f12df440'},
 {'Key': 'PipelineNamePrefix', 'Value': 's3-fs-ingest-pipeline'}]

마지막으로 서비스 카탈로그 제품 템플릿에서 SageMaker 프로젝트를 생성합니다.

In [11]:
# create SageMaker project
r = sm.create_project(
    ProjectName=project_name,
    ProjectDescription="Feature Store ingestion from S3",
    ServiceCatalogProvisioningDetails={
        'ProductId': product_id,
        'ProvisioningArtifactId': provisioning_artifact_ids,
        'ProvisioningParameters': project_parameters
    },
)

print(r)
project_id = r["ProjectId"]

{'ProjectArn': 'arn:aws:sagemaker:ap-northeast-2:018763497627:project/s3-fs-ingest-07-11-15-27', 'ProjectId': 'p-opu00heyaxaa', 'ResponseMetadata': {'RequestId': '5680400e-1d3a-4a04-af13-ef428380f0c5', 'HTTPStatusCode': 200, 'HTTPHeaders': {'x-amzn-requestid': '5680400e-1d3a-4a04-af13-ef428380f0c5', 'content-type': 'application/x-amz-json-1.1', 'content-length': '124', 'date': 'Mon, 07 Feb 2022 11:15:29 GMT'}, 'RetryAttempts': 0}}


<div class="alert alert-info"> 💡 <strong> 프로젝트 생성이 완료될 때까지 기다리기 </strong>
</div>
클라우드 포메이션 콘솔로 이동해서 확인하셔도 되고, 세이지 메이커 프로젝트 화면에서 확인도 가능합니다.

![cf-project.png](img/cf-project.png)

### 옵션 2 섹션 끝
---

# 5. 세이지 메이커 사용자 템플릿을 통한 프로젝트 분석 (데이타 수집 프로젝트)
- 아래의 스튜디오 화면은 위의 UI 에서 프로젝트 생성으로 방식으로 들어가시면 됩니다. 
- 이미 옵션2를 실행 하셨으면 Project 한 개가 보입니다. 클릭하시고 보세요.

## 5.1. 프로젝트 템플릿은 자동화된 데이터 변환 및 수집에 필요한 모든 리소스를 생성
- 지정된 S3 접두사에 새 데이터가 업로드될 때마다 AWS Lambda 함수를 시작하기 위한 [EventBridge 규칙](https://docs.aws.amazon.com/eventbridge/latest/userguide/eb-rules.html)
- SageMaker 파이프라인을 시작하는 AWS Lambda 함수
- DataWrangler 프로세서를 사용하여 처리 작업을 실행하는 SageMaker 파이프라인
- 업로드된 '.flow' 파일을 데이터 변환 워크플로와 함께 사용하는 DataWrangler 프로세서

## 5.2 시드 코드가 있는 CodeCommit 리포지토리
- 파이프라인 생성 및 파이프라인 파라미터 구성을 위한 모든 소스 코드는 [AWS CodeCommit](https://aws.amazon.com/codecommit/) 리포지토리로 제공됩니다. 
    - 코드는 완벽하게 작동하며 즉시 사용할 수 있습니다. 이 코드를 소유하고 요구 사항에 따라 파이프라인의 구성 또는 매개변수를 변경할 수 있습니다.
- 코드 작업을 시작하려면 저장소를 Studio 사용자의 홈 디렉토리에 복제해야 합니다.

<img src="../img/studo-project-clone-repo.png" alt="studo-project-clone-repo" width="800"/>

- 소스 코드를 변경하고 CodeCommit 리포지토리로 푸시할 수 있습니다. 프로젝트는 또한 [AWS CodeBuild](https://aws.amazon.com/codebuild/) 단계를 시작하는 [AWS CodePipeline](https://aws.amazon.com/codepipeline/) CI/CD 파이프라인을 제공합니다. 
- 저장소에 새 커밋이 있을때 마다 빌드는 저장소에서 코드를 가져오고 `create_pipeline` 함수(파일 `build.py`)를 호출합니다. 
- 기존 코드를 변경하거나 `pipeline.py` 파일의 `pipeline.create_pipeline`에 고유한 코드를 제공할 수 있습니다. 
    - 경로: amazon-sagemaker-reusable-components-kr/project-seed-code/s3-fs-ingestion/pipeline.py
```
    # create DW processor
    processor = Processor(
        role=execution_role,
        image_uri=container_uri,
        instance_count=p_processing_instance_count,
        instance_type=p_processing_instance_type,
        volume_size_in_gb=p_processing_volume_size,
        sagemaker_session=sagemaker_session,
    )

    step_process = ProcessingStep(
        name="datawrangler-processing-to-feature-store",
        processor=processor,
        inputs=[flow_input] + [data_input],
        outputs=[processing_job_output],
        job_arguments=[f"--output-config '{json.dumps(output_config)}'"],
    )

    pipeline = Pipeline(
        name=pipeline_name,
        parameters=[
            p_processing_instance_type, 
            p_processing_instance_count,
            p_processing_volume_size,
            p_flow_output_name,
            p_input_flow,
            p_input_data,
            p_feature_group_name
        ],
        steps=[step_process],
        sagemaker_session=sagemaker_session
    )

    response = pipeline.upsert(
        role_arn=execution_role,
        description=pipeline_description,
        tags=[
        {'Key': 'sagemaker:project-name', 'Value': project_name },
        {'Key': 'sagemaker:project-id', 'Value': project_id }
    ],
    )
```

- 기본 코드는 Data Wrangler 프로세서를 사용하여 SageMaker 파이프라인을 구성하고 파이프라인을 upserts합니다.

## 5.3. SageMaker 파이프라인
- 이 프로젝트는 Data Wrangler 프로세서를 사용한 하나의 처리 단계로 구성된 SageMaker 파이프라인을 제공합니다. 
- 파이프라인은 지정된 Data Wrangler `.flow` 파일에 포함된 변환을 수행하고 피쳐 저장소의 지정된 피쳐 그룹에서 변환된 피쳐를 수집합니다.
- 이 파이프라인은 지정된 S3 위치에 새 파일이 업로드될 때마다 Lambda 함수에 의해 시작됩니다. 
- 파이프라인은 프로젝트에 연결되어 있으며 프로젝트 세부정보 페이지의 **파이프라인** 탭에서 사용할 수 있습니다.

<img src="../img/studio-project-details-pipelines.png" alt="studio-project-details-pipelines" width="800"/>

여기에서 파이프라인 그래프, 매개변수, 설정 및 실행 기록을 볼 수 있습니다.

<img src="../img/studio-pipeline-execution-history.png" alt="studio-pipeline-execution-history" width="800"/>


**실행 시작**을 클릭하고 파이프라인 매개변수를 제공하여 Studio에서 수동으로 새 실행을 시작할 수도 있습니다.

<img src="../img/studio-pipeline-parameter-input.png" alt="studio-pipeline-parameter-input" width="500"/>

# 6. 자동화 파이프라인 테스트

배포된 데이터 변환 및 피쳐 저장소 수집 파이프라인을 테스트하려면 다음 단계를 수행하십시오.
1. 모니터링되는 S3 접두사 위치에 데이터 파일을 업로드합니다. 그러면 데이터 파이프라인을 통해 데이터 변환 및 수집이 시작됩니다.
1. 파이프라인 실행 모니터링
1. 피쳐 그룹에 로드된 데이터 확인

⭐ EventBridge 규칙은 'PutObject' 및 'CompleteMultipartUpload'라는 두 가지 S3 이벤트를 모니터링합니다. 두 S3 버킷 간에 객체를 복사하면 EventBrige 규칙이 시작되지 않습니다.

다음 S3 `PUT` 이벤트는 새 파이프라인 실행을 시작하는 Lambda 함수를 시작합니다.
![S3-eventbridge-rule.png](img/S3-eventbridge-rule.png)

람다 함수는 파이프라인을 실행 함
![run-pipeline-lambda.png](img/run-pipeline-lambda.png)

## 6.1. 데이터를 S3 버킷에 업로드

In [ ]:
file_name = f"abalone-{strftime('%d-%H-%M-%S', gmtime())}.csv"

In [ ]:
!aws s3 cp {abalone_dataset_local_url} s3://{s3_input_data_prefix}{file_name}

## 6.2. 파이프라인 실행을 모니터링

In [ ]:
try:
    project_id
    project_name
except NameError:
    raise Exception("[ERROR]: project_id or project_name variables are not set")
    
if project_id is None or project_name is None:
    raise Exception("[ERROR]: project_id or project_name variables are not set")

In [ ]:
# Get the the project data
r = sm.describe_project(ProjectName=project_name)

# Get the pipeline prefix from the project parameters
pipeline_name_prefix = [p for p in r["ServiceCatalogProvisioningDetails"]["ProvisioningParameters"] if p["Key"] == "PipelineNamePrefix"][0]["Value"]

In [ ]:
pipeline_name_prefix

In [ ]:
# set the pipeline name
s3_to_fs_pipeline_name = f"{pipeline_name_prefix}-{project_id}"

%store s3_to_fs_pipeline_name

In [ ]:
# check pipeline execution 
summaries = sm.list_pipeline_executions(PipelineName=s3_to_fs_pipeline_name).get('PipelineExecutionSummaries')
summaries

In [ ]:
latest_execution = sm.list_pipeline_executions(PipelineName=s3_to_fs_pipeline_name).get('PipelineExecutionSummaries')[0].get('PipelineExecutionArn')
print (latest_execution)

In [ ]:
# Wait for pipeline execution to complete 'Executing' status
while sm.describe_pipeline_execution(PipelineExecutionArn=latest_execution)["PipelineExecutionStatus"] == "Executing":
    print('Pipeline is in Executing status...')
    time.sleep(30)
    
print('Pipeline is done Executing')
print(sm.describe_pipeline_execution(PipelineExecutionArn=latest_execution))

또는 Studio의 파이프라인 위젯 내에서 파이프라인 실행을 모니터링할 수 있습니다.

![](../img/studio-pipeline-executing.png)

## 7. 피쳐 스토어에서 데이터 확인
실행이 완료되면 데이터가 피쳐 그룹에 로드되었는지 확인할 수 있습니다.

피쳐 그룹 오브젝트를 생성

In [ ]:
feature_store_session = Session()

feature_group = FeatureGroup(
    name=feature_group_name, 
    sagemaker_session=feature_store_session
)

In [ ]:
# Build SQL query to features group
fs_query = feature_group.athena_query()

query_string = f'SELECT * FROM "{fs_query.table_name}"'
print(f'Prepared query {query_string}')
print(fs_query)

In [ ]:
# Run Athena query. The output is loaded to a Pandas dataframe.
fs_query.run(
    query_string=query_string, 
    output_location=f"s3://{s3_fs_query_output_prefix}"
)

fs_query.wait()
data_df = fs_query.as_dataframe()

The `DataFrame` contains now all features from the feature group:

In [ ]:
data_df

## 8. Next
 [`99-clean-up` notebook](99-clean-up.ipynb) 로 이동하셔서 리소스를 제거 하세요.

# A. 에러 케이스 및 트러블 슈팅

## A.1. Project 실행시 에러

![lambda_error.png](../img2/lambda_error.png)


## 해결
- AmazonSageMakerServiceCatalogProductsLaunchRole 에 LambdaFull 추가 함


## A.2 랭글러 에러
- 원본의 에러 발생은 데이터 소스가 달라서 밠생 함.
- [해결] 직접 데이터 랭글러 데이터 플로우를 생성하여 해결 함.
![wrangler_erro.png](../img2/wrangler_error.png)